<a href="https://colab.research.google.com/github/thanksbinh/student-posts-classifier/blob/main/StudentRequestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install py_vncorenlp

In [12]:
!pip install emoji

In [1]:
# Import Basis Libraries
import pandas as pd

df = pd.read_csv('./test_data_01.csv')

# rename column
# df.rename(columns = {'label (depression result)':'Sentiment'}, inplace = True)
# df.rename(columns = {'message to examine':'review'}, inplace = True)

# Drop Columns
# df.drop('text',axis=1,inplace=True) # Text is post_text with header

# Drop rows
df.drop_duplicates(subset='post_id', inplace=True)
df.drop(df[df.post_text != df.post_text].index, inplace=True)
# df['post_text'] = df['post_text'].fillna('')

# Lowercase
df['post_text'] = df['post_text'].str.lower()
df.head()

,post_id,post_text,shared_text,original_text,time,image_lowquality,images_lowquality,images_lowquality_description,likes,comments,shares,link,username,user_url,is_live,reaction_count,with,was_live,header
1,906358048077279,cần pass lại 2 quyển này ạ,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', ""May be an image o...",NaN,0,0,NaN,Anonymous participant,NaN,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Anonymous participant > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
4,906402471406170,môn học nào em thấy khó nhất nhỉ?,NaN,NaN,2024-10-09 10:00:00,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg4-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', None, None, None]",355.0,13,0,NaN,Thanh Ngọc,https://facebook.com/profile.php?id=6155790119...,False,355.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Thanh Ngọc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
5,905552854824465,nv1 của các cậu là trường nào?,NaN,NaN,2024-10-08 18:03:00,https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6/A...,['https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6...,"[None, None, None]",33.0,25,0,NaN,Gạo Luộc,https://facebook.com/profile.php?id=1000930172...,False,33.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Gạo Luộc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
6,906378388075245,vào ib tớ nhé,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of 1 person and text that sa...",NaN,3,0,NaN,Ng Thuyy Lynhh,https://facebook.com/profile.php?id=1000900553...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Ng Thuyy Lynhh > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
7,906315738081510,điểm thi giữa kì của các tháng😇😇,NaN,NaN,2024-10-09 08:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of text that says 'ĐIỂM TOÁN...",NaN,24,0,NaN,Nhật Hạ,https://facebook.com/profile.php?id=6156512453...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Nhật Hạ > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP


In [2]:
len(df)

4723

In [16]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local working folder
# py_vncorenlp.download_model(save_dir="/content/vncorenlp")

# Load VnCoreNLP from the local working folder that contains both `VnCoreNLP-1.2.jar` and `models`
model = py_vncorenlp.VnCoreNLP(save_dir='/content/vncorenlp')
# Equivalent to: model = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos", "ner", "parse"], save_dir='./vncorenlp')

# Annotate a raw corpus
# model.annotate_file(input_file="/content/input_file", output_file="/content/output_file")

# Annotate a raw text
# model.print_out(model.annotate_text("Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."))

def word_segment_wrapper(text):
  return " ".join(model.word_segment(text))

1	Ông	Nc	O	4	sub
2	Nguyễn_Khắc_Chúc	Np	B-PER	1	nmod
3	đang	R	O	4	adv
4	làm_việc	V	O	0	root
5	tại	E	O	4	loc
6	Đại_học	N	B-ORG	5	pob
7	Quốc_gia	N	I-ORG	6	nmod
8	Hà_Nội	Np	I-ORG	6	nmod
9	.	CH	O	4	punct

1	Bà	Nc	O	9	sub
2	Lan	Np	B-PER	1	nmod
3	,	CH	O	1	punct
4	vợ	N	O	1	nmod
5	ông	Nc	O	4	nmod
6	Chúc	Np	B-PER	5	nmod
7	,	CH	O	1	punct
8	cũng	R	O	9	adv
9	làm_việc	V	O	0	root
10	tại	E	O	9	loc
11	đây	P	O	10	pob
12	.	CH	O	9	punct



In [38]:
import re
import string

# Storing Punctuation in a Variable
punc = string.punctuation.replace("?", "")
punc

'!"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'

In [3]:
# Function to remove HTML Tags
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

# Here We also Use Regular Expressions to Remove URLs from Text or Whole Corpus.
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

# The code defines a function, remove_punc1, that takes a text input and removes all punctuation characters from it using
# the translate method with a translation table created by str.maketrans. This function effectively cleanses the text of punctuation symbols.
def remove_punc(text):
    return text.translate(str.maketrans('', '', punc))

In [4]:
df['post_text'] = df['post_text'].apply(remove_url)
df['post_text'] = df['post_text'].apply(remove_punc)
df.head()

,post_id,post_text,shared_text,original_text,time,image_lowquality,images_lowquality,images_lowquality_description,likes,comments,shares,link,username,user_url,is_live,reaction_count,with,was_live,header
1,906358048077279,cần pass lại 2 quyển này ạ,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', ""May be an image o...",NaN,0,0,NaN,Anonymous participant,NaN,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Anonymous participant > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
4,906402471406170,môn học nào em thấy khó nhất nhỉ,NaN,NaN,2024-10-09 10:00:00,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg4-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', None, None, None]",355.0,13,0,NaN,Thanh Ngọc,https://facebook.com/profile.php?id=6155790119...,False,355.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Thanh Ngọc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
5,905552854824465,nv1 của các cậu là trường nào,NaN,NaN,2024-10-08 18:03:00,https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6/A...,['https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6...,"[None, None, None]",33.0,25,0,NaN,Gạo Luộc,https://facebook.com/profile.php?id=1000930172...,False,33.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Gạo Luộc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
6,906378388075245,vào ib tớ nhé,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of 1 person and text that sa...",NaN,3,0,NaN,Ng Thuyy Lynhh,https://facebook.com/profile.php?id=1000900553...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Ng Thuyy Lynhh > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
7,906315738081510,điểm thi giữa kì của các tháng😇😇,NaN,NaN,2024-10-09 08:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of text that says 'ĐIỂM TOÁN...",NaN,24,0,NaN,Nhật Hạ,https://facebook.com/profile.php?id=6156512453...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Nhật Hạ > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP


In [19]:
df['post_text'] = df['post_text'].apply(word_segment_wrapper)
df.head()

,post_id,post_text,shared_text,original_text,time,image_lowquality,images_lowquality,images_lowquality_description,likes,comments,shares,link,username,user_url,is_live,reaction_count,with,was_live,header
1,906358048077279,cần pass lại 2 quyển này ạ,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', ""May be an image o...",NaN,0,0,NaN,Anonymous participant,NaN,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Anonymous participant > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
4,906402471406170,môn_học nào em thấy khó nhất nhỉ,NaN,NaN,2024-10-09 10:00:00,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg4-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', None, None, None]",355.0,13,0,NaN,Thanh Ngọc,https://facebook.com/profile.php?id=6155790119...,False,355.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Thanh Ngọc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
5,905552854824465,nv1 của các cậu là trường nào,NaN,NaN,2024-10-08 18:03:00,https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6/A...,['https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6...,"[None, None, None]",33.0,25,0,NaN,Gạo Luộc,https://facebook.com/profile.php?id=1000930172...,False,33.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Gạo Luộc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
6,906378388075245,vào inbox tớ nhé,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of 1 person and text that sa...",NaN,3,0,NaN,Ng Thuyy Lynhh,https://facebook.com/profile.php?id=1000900553...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Ng Thuyy Lynhh > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
7,906315738081510,điểm thi giữa kì của các thángsmilingfacewithh...,NaN,NaN,2024-10-09 08:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of text that says 'ĐIỂM TOÁN...",NaN,24,0,NaN,Nhật Hạ,https://facebook.com/profile.php?id=6156512453...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Nhật Hạ > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP


In [6]:
# prompt: import vietnamese_stopwords from ./vietnamese-stopwords-dash.txt

with open('./vietnamese-stopwords-dash.txt', 'r', encoding='utf-8') as f:
  vietnamese_stopwords = [line.strip() for line in f]

print(vietnamese_stopwords)

def remove_stopwords(text):
    new_text = []
    for i in text.split():
        if i not in vietnamese_stopwords:
            new_text.append(i)
    return " ".join(new_text)

In [8]:
# Here Come ChatWords Which i Get from a Github Repository
# Repository Link : https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt
# https://howtovietnamese.com/vietnamese-text-abbreviations-slang/
chat_words = {
    "a": "anh",
    "c": "chị",
    "cj": "chị",
    "e": "em",
    "ae": "anh_em",
    "ce": "chị_em",
    "ace": "anh_chị_em",
    "b": "bạn",
    "t": "tôi",
    # "m": "mày",
    "m": "mình",
    "mik": "mình",
    "mng": "mọi_người",
    "mn": "mọi_người",
    "ng": "người",

    "iu": "yêu",
    "ct": "chia_tay",
    "ny": "người_yêu",
    "vk": "vợ",
    "ck": "chồng",

    "h": "giờ",
    "g": "giờ",
    "p": "phút",
    # "bh": "bây giờ",
    # "bh": "bao giờ",
    "hqua": "hôm_qua",
    "htrc": "hôm_trước",

    "ib": "inbox",
    "lb": "inbox",
    "ibox": "inbox",
    "nt": "nhắn_tin",
    # "nc": "nói chuyện",
    # "nc": "nói chung",
    # "nch": "nói chuyện",
    # "nch": "nói chung",
    "nchuyen": "nói_chuyện",
    "mún": "muốn",
    "bít": "biết",
    "bik": "biết",
    "cb": "chuẩn_bị",

    "dt": "điện_thoại",
    "đt": "điện_thoại",
    "bt": "bình_thường",
    "bth": "bình_thường",

    "j": "gì",
    "b": "bạn",
    "bn": "bạn",
    "bb": "bạn_bè",
    # "bn": "bao nhiêu",
    "ntn": "như_thế_nào",

    "kh": "không",
    "k": "không",
    "khong": "không",
    "ko": "không",
    "hem": "không",
    "hong": "không",
    "khum": "không",
    # "hok": "không",
    "đc": "được",
    "dc": "được",
    "r": "rồi",
    "nhìu": "nhiều",
    "wá": "quá",
    "cx": "cũng",
    "uk": "ừ",
    "uh": "ừ",
    "ukm": "ừ",
    "g9": "ngủ_ngon",
    "vn": "việt_nam",

    "lquan": "liên_quan",
    "cs": "có",
    "ah": "à",
    "vs": "với",
    # "sn": "sinh năm",
    # "sn": "sinh nhật",
    "ak": "à",
    "ah": "ạ",
    "cmt": "comment",
    "s": "sao",
    "cgi": "cái gì",
    "v": "vậy",
    "lun": "luôn",
    "zới": "với",
    "gr": "group",
    "sl": "số_lượng",
    "ttv": "tuyển_thành_viên",
    "tv": "thành_viên",
    # "rcm": "recomment",
    "zl": "zalo",
    "lh": "liên_hệ",
    "ad": "admin",
    "sđt": "số điện thoại",
    "đg": "đang",

    # "hk1": "học kỳ 1",
    # "hk2": "học kỳ 2",
    # "nv1": "nguyện vọng 1",
    # "khtn": "khoa học tự nhiên",
    # "nlxh": "nghị luận xã hội",
    "hsg": "học_sinh_giỏi",
    "c3": "cấp_3",
    "lp": "lớp",
    "tl": "tài_liệu",
    # "hok": "học",
    "hc": "học",
    "hs": "học_sinh",
    "sv": "sinh_viên",
    "gv": "giáo_viên",
    "onl": "online",
    "off": "offline",
    "sp": "sư_phạm",
    "nn": "ngoại_ngữ",
}

In [9]:
# Function
def chat_conversion(text):
    new_text = []
    for i in text.split():
        if i in chat_words:
            new_text.append(chat_words[i])
        else:
            new_text.append(i)
    return " ".join(new_text)

In [10]:
# Chat conversion then remove stopwords, because stopwords are usually get shortened

df['post_text'] = df['post_text'].apply(chat_conversion)
df['post_text'] = df['post_text'].apply(remove_stopwords)
df

,post_id,post_text,shared_text,original_text,time,image_lowquality,images_lowquality,images_lowquality_description,likes,comments,shares,link,username,user_url,is_live,reaction_count,with,was_live,header
1,906358048077279,cần pass lại 2 quyển này ạ,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', ""May be an image o...",NaN,0,0,NaN,Anonymous participant,NaN,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Anonymous participant > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
4,906402471406170,môn học nào em thấy khó nhất nhỉ,NaN,NaN,2024-10-09 10:00:00,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg4-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', None, None, None]",355.0,13,0,NaN,Thanh Ngọc,https://facebook.com/profile.php?id=6155790119...,False,355.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Thanh Ngọc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
5,905552854824465,nv1 của các cậu là trường nào,NaN,NaN,2024-10-08 18:03:00,https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6/A...,['https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6...,"[None, None, None]",33.0,25,0,NaN,Gạo Luộc,https://facebook.com/profile.php?id=1000930172...,False,33.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Gạo Luộc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
6,906378388075245,vào inbox tớ nhé,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of 1 person and text that sa...",NaN,3,0,NaN,Ng Thuyy Lynhh,https://facebook.com/profile.php?id=1000900553...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Ng Thuyy Lynhh > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
7,906315738081510,điểm thi giữa kì của các tháng😇😇,NaN,NaN,2024-10-09 08:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of text that says 'ĐIỂM TOÁN...",NaN,24,0,NaN,Nhật Hạ,https://facebook.com/profile.php?id=6156512453...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Nhật Hạ > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016,1446216142928013,góc phát vía thành tích sương sương của học tr...,NaN,NaN,2024-06-18 20:51:00,https://scontent.fsgn2-8.fna.fbcdn.net/v/t39.3...,['https://scontent.fsgn2-8.fna.fbcdn.net/v/t39...,"['No photo description available.', 'No photo ...",5.0,1,0,NaN,Học Lý Thầy Thông,https://facebook.com/lythaythong?refid=18&_ft_...,False,5.0,"[{'name': 'Gia sư tiếng anh', 'link': 'https:/...",False,Học Lý Thầy Thông > ‎Gia sư tiếng anh
7017,1446216622927965,cải thiện điểm số là điều 2k6 cần nhất trong n...,NaN,NaN,2024-06-18 20:52:00,NaN,[],[],NaN,0,0,NaN,Tôi yêu Vật Lý,https://facebook.com/tyvl68?refid=18&_ft_=encr...,False,NaN,"[{'name': 'Gia sư tiếng anh', 'link': 'https:/...",False,Tôi yêu Vật Lý > ‎Gia sư tiếng anh
7019,1445608502988777,cải thiện điểm số là điều 2k6 cần nhất trong n...,NaN,NaN,2024-06-17 21:25:00,https://scontent.fsgn2-3.fna.fbcdn.net/m1/v/t6...,['https://scontent.fsgn2-3.fna.fbcdn.net/m1/v/...,[None],1.0,0,0,NaN,Tôi yêu Vật Lý,https://facebook.com/tyvl68?refid=18&_ft_=encr...,False,1.0,"[{'name': 'Gia sư tiếng anh', 'link': 'https:/...",False,Tôi yêu Vật Lý > ‎Gia sư tiếng anh
7020,1445583329657961,những ngày cuối 2k6 nên ôn gì và ôn như thế nà...,NaN,NaN,2024-06-17 20:30:00,https://scontent.fsgn2-6.fna.fbcdn.net/v/t39.3...,['https://scontent.fsgn2-6.fna.fbcdn.net/v/t39...,"['No photo description available.', 'No photo ...",1.0,0,0,NaN,Học Lý Thầy Thông,https://facebook.com/lythaythong?refid=18&_ft_...,False,1.0,"[{'name': 'Gia sư tiếng anh', 'link': 'https:/...",False,Học Lý Thầy Thông > ‎Gia sư tiếng anh


In [11]:
# Import this Library to Handle the Spelling Issue.
# from textblob import TextBlob

# # Incorrect text
# incorrect_text = 'ceertain conditionas duriing seveal ggenerations aree moodified in the saame maner.'
# print(incorrect_text)
# # Text 2
# incorrect_text2 = 'The cat sat on the cuchion. while plyaiing'
# # Calling function
# textBlb = TextBlob(incorrect_text)
# textBlb1 = TextBlob(incorrect_text2)
# # Corrected Text
# print(textBlb.correct().string)
# print(incorrect_text2)
# print(textBlb1.correct().string)

# Todo: Do the above, but for Vietnamese

In [13]:
# We will USe the Emoji Libray to handle this task
# Pip Install emoji
import emoji

In [14]:
# Calling the Emoji tool Demojize.
df['post_text'] = df['post_text'].apply(emoji.demojize)
df['post_text'] = df['post_text'].apply(remove_punc) # Remove :
df.head()

,post_id,post_text,shared_text,original_text,time,image_lowquality,images_lowquality,images_lowquality_description,likes,comments,shares,link,username,user_url,is_live,reaction_count,with,was_live,header
1,906358048077279,cần pass lại 2 quyển này ạ,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', ""May be an image o...",NaN,0,0,NaN,Anonymous participant,NaN,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Anonymous participant > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
4,906402471406170,môn học nào em thấy khó nhất nhỉ,NaN,NaN,2024-10-09 10:00:00,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg4-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', None, None, None]",355.0,13,0,NaN,Thanh Ngọc,https://facebook.com/profile.php?id=6155790119...,False,355.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Thanh Ngọc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
5,905552854824465,nv1 của các cậu là trường nào,NaN,NaN,2024-10-08 18:03:00,https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6/A...,['https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6...,"[None, None, None]",33.0,25,0,NaN,Gạo Luộc,https://facebook.com/profile.php?id=1000930172...,False,33.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Gạo Luộc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
6,906378388075245,vào inbox tớ nhé,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of 1 person and text that sa...",NaN,3,0,NaN,Ng Thuyy Lynhh,https://facebook.com/profile.php?id=1000900553...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Ng Thuyy Lynhh > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP
7,906315738081510,điểm thi giữa kì của các thángsmilingfacewithh...,NaN,NaN,2024-10-09 08:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of text that says 'ĐIỂM TOÁN...",NaN,24,0,NaN,Nhật Hạ,https://facebook.com/profile.php?id=6156512453...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Nhật Hạ > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP


In [20]:
# Import Libraray
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize,sent_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
df['tokens'] = df['post_text'].apply(word_tokenize)
df.head()

,post_id,post_text,shared_text,original_text,time,image_lowquality,images_lowquality,images_lowquality_description,likes,comments,shares,link,username,user_url,is_live,reaction_count,with,was_live,header,tokens
1,906358048077279,cần pass lại 2 quyển này ạ,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', ""May be an image o...",NaN,0,0,NaN,Anonymous participant,NaN,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Anonymous participant > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP,"[cần, pass, lại, 2, quyển, này, ạ]"
4,906402471406170,môn_học nào em thấy khó nhất nhỉ,NaN,NaN,2024-10-09 10:00:00,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg4-1.xx.fbcdn.net/v/t39.3...,"['May be an image of text', None, None, None]",355.0,13,0,NaN,Thanh Ngọc,https://facebook.com/profile.php?id=6155790119...,False,355.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Thanh Ngọc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP,"[môn_học, nào, em, thấy, khó, nhất, nhỉ]"
5,905552854824465,nv1 của các cậu là trường nào,NaN,NaN,2024-10-08 18:03:00,https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6/A...,['https://scontent-hkg4-1.xx.fbcdn.net/m1/v/t6...,"[None, None, None]",33.0,25,0,NaN,Gạo Luộc,https://facebook.com/profile.php?id=1000930172...,False,33.0,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Gạo Luộc > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP,"[nv1, của, các, cậu, là, trường, nào]"
6,906378388075245,vào inbox tớ nhé,NaN,NaN,2024-10-09 22:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of 1 person and text that sa...",NaN,3,0,NaN,Ng Thuyy Lynhh,https://facebook.com/profile.php?id=1000900553...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Ng Thuyy Lynhh > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP,"[vào, inbox, tớ, nhé]"
7,906315738081510,điểm thi giữa kì của các thángsmilingfacewithh...,NaN,NaN,2024-10-09 08:00:00,https://scontent-hkg1-2.xx.fbcdn.net/v/t39.308...,['https://scontent-hkg1-2.xx.fbcdn.net/v/t39.3...,"[""May be an image of text that says 'ĐIỂM TOÁN...",NaN,24,0,NaN,Nhật Hạ,https://facebook.com/profile.php?id=6156512453...,False,NaN,"[{'name': '2K10 ĐẤU TRƯỜNG HỌC TẬP', 'link': '...",False,Nhật Hạ > ‎2K10 ĐẤU TRƯỜNG HỌC TẬP,"[điểm, thi, giữa, kì, của, các, thángsmilingfa..."


In [23]:
from collections import defaultdict

dictionary = defaultdict(int)

def extract_dictionary_paddle(token):
    dictionary[token] += 1

    return dictionary

In [24]:
for i, row in df.iterrows():
    tokens = row['tokens']
    for token in tokens:
        extract_dictionary_paddle(token)

In [25]:
dicDf = pd.DataFrame(dictionary.items(), columns=['word', 'count'])
dicDf = dicDf[dicDf['count'] > 1]
dicDf.sort_values(by=['count'], ascending=False)

,word,count
92,anh,6219
53,mình,1650
33,có,1595
88,học,1551
6,ạ,1542
...,...,...
4703,knao,1
4702,vĩnh_phúc,1
4701,vĩnh_yên,1
4700,haahn,1


In [26]:
# dicDf.to_csv('dicDf.csv', index=False)

In [33]:
dicDf.sort_values(by=['count'], ascending=False)

,word,count
92,anh,6219
53,mình,1650
33,có,1595
88,học,1551
6,ạ,1542
...,...,...
4685,lớppp,1
4684,chức_vụ,1
4683,nhéfourleafcloverfourleafcloverfourleafclover,1
4682,tràn_ngập,1
